In [1]:
!git clone https://github.com/AbdulDD/DocVQA-Tasks.git
%cd DocVQA-Tasks/Llava

Cloning into 'DocVQA-Tasks'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 48 (delta 17), reused 26 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 44.36 KiB | 4.93 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/DocVQA-Tasks/Llava


In [2]:
!pip install -r requirements.txt
!pip install pytorch-lightning

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.5 MB/s eta 0:00:00
  Created wheel for lint: filename=lint-1.2.1-py3-none-any.whl size=17771 sha256=4800a7487b6150a8ce4251bd17395b18a6bfb6057b06e1b04db7eed8fa263415
  Stored in directory: /root/.cache/pip/wheels/2c/05/e4/bfc01de0be087a6a87c1ff96c255c9fd0cc45ca8cbf1ac63c1
Successfully built lint
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.

In [1]:
# Imports
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoProcessor, LlavaForConditionalGeneration

In [2]:
# 1. Load processor
processor = AutoProcessor.from_pretrained('llava-hf/llava-1.5-7b-hf')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [5]:
# 2. Load custom dataset class and process data
from llavadataset import LlavaDataset
dataset = LlavaDataset("AbdulMuqtadir/Doc_VQA_subset", processor)

README.md:   0%|          | 0.00/774 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.19M [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/5.78M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/9.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

In [6]:
# Check Shapes
sample = dataset[3]

input_ids = sample["input_ids"]
attention_mask = sample["attention_mask"]
pixel_values = sample["pixel_values"]
labels = sample["labels"]

print(input_ids.shape, attention_mask.shape, pixel_values.shape, labels.shape)


torch.Size([1024]) torch.Size([1024]) torch.Size([3, 336, 336]) torch.Size([1024])


In [7]:
# Check attention mask tokens

print("Attention mask:")
print(attention_mask)
print("\nSum (prompt length):", attention_mask.sum().item())
print("\n" + "-"*60)


Attention mask:
tensor([0, 0, 0,  ..., 1, 1, 1])

Sum (prompt length): 602

------------------------------------------------------------


In [8]:
# Load DataLoader and call training dataset
from torch.utils.data import DataLoader
training_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
training_dataloader

In [9]:
for batch_idx, batch in enumerate(training_dataloader):
    input_ids = batch["input_ids"]         # shape: [batch_size, 1024]
    attention_mask = batch["attention_mask"]  # shape: [batch_size, 1024]
    pixel_values = batch["pixel_values"]      # shape: [batch_size, 3, 336, 336]
    labels = batch["labels"]                 # shape: [batch_size, 1024]

    print("input_ids shape:", input_ids.shape)
    print("attention_mask shape:", attention_mask.shape)
    print("pixel_values shape:", pixel_values.shape)
    print("labels shape:", labels.shape)

    break  # only look at the first batch


input_ids shape: torch.Size([4, 1024])
attention_mask shape: torch.Size([4, 1024])
pixel_values shape: torch.Size([4, 3, 336, 336])
labels shape: torch.Size([4, 1024])


In [10]:
# Load model
model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf", load_in_8bit=True, dtype=torch.float16)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [11]:
# Imports
import pytorch_lightning as pl

class LlavaTraining(pl.LightningModule):
    def __init__(self, config, model, processor):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model
        self.model.train()

    def training_step(self, batch, batch_idx):

        # 1. Extract the inputs
        input_ids = batch["input_ids"]         # [B, 1024]
        attention_mask = batch["attention_mask"]  # [B, 1024]
        pixel_values = batch["pixel_values"]      # [B, 3, 336, 336]
        labels = batch["labels"]                 # [B, 1024]

        # 2. Forward pass
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            labels=labels
        )

        loss = outputs.loss

        # 3. Log the loss
        self.log(
            "train_loss",
            loss,
            on_step=True,
            on_epoch=True,
            prog_bar=True,
            logger=True
        )

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=self.config["lr"]   # FIXED
        )
        return optimizer


In [14]:
config = {
    "lr": 0.001,
    "max_epochs": 10,
}



lightning_model = LlavaTraining(config=config, model=model, processor=processor)

trainer = pl.Trainer(
    max_epochs=config["max_epochs"],
    precision="32-true",
    log_every_n_steps=1,
    gradient_clip_val=1.0,
    #accelerator="gpu",
)

trainer.fit(lightning_model, training_dataloader)


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


OutOfMemoryError: CUDA out of memory. Tried to allocate 44.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 40.12 MiB is free. Process 25502 has 14.70 GiB memory in use. Of the allocated memory 14.30 GiB is allocated by PyTorch, and 265.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)